# BigQuery query magic

Jupyter magics are notebook-specific shortcuts that allow you to run commands with minimal syntax. Jupyter notebooks come with many [built-in commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html). The BigQuery client library, `google-cloud-bigquery`, provides a cell magic, `%%bigquery`. The `%%bigquery` magic runs a SQL query and returns the results as a pandas `DataFrame`.

## Run a query on a public dataset

The following example queries the BigQuery `usa_names` public dataset. `usa_names` is a Social Security Administration dataset that contains all names from Social Security card applications for births that occurred in the United States after 1879.

The following example shows how to invoke the magic (`%%bigquery`), and how to pass in a standard SQL query in the body of the code cell. The results are displayed below the input cell as a pandas [`DataFrame`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
from google.oauth2 import service_account
import google.oauth2.credentials
import json
from google_auth_oauthlib.helpers import credentials_from_session

%load_ext google.cloud.bigquery

#credentials = google.oauth2.credentials.Credentials('access_token')

In [7]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=ydTeSC0AfGUzMbMrz6g8n6tJwIlQ7v&access_type=offline&code_challenge=EWGJbKpuLU_DZ2-WOcRaJZSKB1kkSa-zl1T6JpDFQP8&code_challenge_method=S256

^C


Command killed by keyboard interrupt



In [ ]:
## copy and paste into terminal

#!gcloud config set project edd-trn-mdai-pr-cah

In [ ]:
sql = """
    SELECT *
    FROM `edd-trn-mdai-pr-cah.D2_ML_MED_CUSTCHRN_NP_EXPM.US_COVID19_CASES_AND_DEATHS_BY_STATE_OVER_TIME `
    LIMIT 10
"""

df = pandas_gbq.read_gbq(sql)

df.head()

## Display verbose output

As the query job is running, status messages below the cell update with the query job ID and the amount of time the query has been running. By default, this output is erased and replaced with the results of the query. If you pass the `--verbose` flag, the output will remain below the cell after query completion.

In [2]:
%%bigquery
SELECT name, SUM(number) as count
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name
ORDER BY count DESC
LIMIT 10

/Users/jcachat/anaconda3/envs/base-gcp/lib/python3.9/site-packages/google/cloud/bigquery/job/query.py:1687: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/jcachat/anaconda3/envs/base-gcp/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2007: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,name,count
0,James,5054074
1,John,4910976
2,Robert,4763102
3,Michael,4396482
4,William,3939912
5,Mary,3757625
6,David,3631489
7,Joseph,2554530
8,Richard,2550606
9,Charles,2299419


## Explicitly specify a project

By default, the `%%bigquery` magic command uses your default project to run the query. You may also explicitly provide a project ID using the `--project` flag. Note that your credentials must have permissions to create query jobs in the project you specify.

In [ ]:
project_id = "your-project-id"

In [ ]:
%%bigquery --project $project_id
SELECT name, SUM(number) as count
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name
ORDER BY count DESC
LIMIT 10

## Assign the query results to a variable

To save the results of your query to a variable, provide a variable name as a parameter to `%%bigquery`. The following example saves the results of the query to a variable named `df`. Note that when a variable is provided, the results are not displayed below the cell that invokes the magic command.

In [ ]:
%%bigquery df
SELECT name, SUM(number) as count
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name
ORDER BY count DESC
LIMIT 10

In [ ]:
df

## Run a parameterized query

Parameterized queries are useful if you need to run a query with certain parameters that are calculated at run time. Note that the value types must be JSON serializable. The following example defines a parameters dictionary and passes it to the `--params` flag. The key of the dictionary is the name of the parameter, and the value of the dictionary is the value of the parameter.

In [ ]:
params = {"limit": 10}

In [ ]:
%%bigquery --params $params
SELECT name, SUM(number) as count
FROM `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY name
ORDER BY count DESC
LIMIT @limit